# kutubxonalarni chaqiramiz

In [1]:
import pandas as pd
import numpy as np

from joblib import dump
import os

from sklearn.model_selection import train_test_split, KFold,cross_val_score
from sklearn.feature_selection import RFE

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from xgboost import XGBClassifier

Xgb_Clas = XGBClassifier()
N_neig = NearestNeighbors()
DT_Clas = DecisionTreeClassifier()
RF_Clas = RandomForestClassifier()
SV_Clas = SVC()
LR_Clas = LogisticRegression()

from sklearn.metrics import accuracy_score

# Datasetni yuklaymiz

In [5]:
df = pd.read_csv(r"C:\Users\bunyo\OneDrive\Desktop\git_project\2_week_project\data\preprocessed_data\preprocessed_data.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22899 entries, 0 to 22898
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   latitude                        22899 non-null  float64
 1   longitude                       22899 non-null  float64
 2   mag                             22899 non-null  float64
 3   clustering_coefficient_30_days  22899 non-null  float64
 4   std_mag_30_days                 22899 non-null  float64
 5   rolling_mean_depth_30_days      22899 non-null  float64
 6   earthquakes_last_30_days        22899 non-null  float64
 7   b_value                         22899 non-null  float64
 8   b_value_increment_i_i2          22899 non-null  float64
 9   b_value_increment_i2_i4         22899 non-null  float64
 10  b_value_increment_i4_i6         22899 non-null  float64
 11  b_value_increment_i6_i8         22899 non-null  float64
 12  b_value_increment_i8_i10        

# Filter bilan eng yaxshi feturelarni olamiz

In [7]:
corr_Df = df.corr()['class']
corr_Df

latitude                          0.028836
longitude                        -0.073590
mag                               0.036746
clustering_coefficient_30_days   -0.005725
std_mag_30_days                   0.024852
rolling_mean_depth_30_days       -0.058665
earthquakes_last_30_days         -0.009482
b_value                          -0.035609
b_value_increment_i_i2            0.000415
b_value_increment_i2_i4           0.000202
b_value_increment_i4_i6           0.000345
b_value_increment_i6_i8           0.000534
b_value_increment_i8_i10          0.000439
max_mag_last_week                 0.036399
eta                               0.037867
delta_M                          -0.007755
elapsed_time                     -0.011789
coefficient_of_variation          0.075612
dE1_2                             0.124231
class                             1.000000
mag_depth_ratio                   0.004803
recent_activity_rate              0.010731
Name: class, dtype: float64

In [9]:
corr_df = corr_Df[abs(corr_Df) > 0.01]
corr_df

latitude                      0.028836
longitude                    -0.073590
mag                           0.036746
std_mag_30_days               0.024852
rolling_mean_depth_30_days   -0.058665
b_value                      -0.035609
max_mag_last_week             0.036399
eta                           0.037867
elapsed_time                 -0.011789
coefficient_of_variation      0.075612
dE1_2                         0.124231
class                         1.000000
recent_activity_rate          0.010731
Name: class, dtype: float64

# kerakli ustunlarni saqlab qoyamiz

In [15]:
df_corr = df[corr_df.index]

# -----------------------------------------------------------------

 # ikki xil yol tutamiz filter methodi bilan va filtersiz model yasaymiz, bunga sabab
 # filter qisak kerakli ustunlarni ham tashab ketyapti, shuning uchun wraper va embeded dan foydalanamiz

# -----------------------------------------------------------------



# datasetni bolamiz(filtersiz)

In [10]:
x = df.drop('class',axis=1)
y = df['class']

# filter ishlatilgan datasetni bolamiz

In [21]:
x1 = df_corr.drop('class',axis=1)
y1 = df_corr['class']

# train test ga ajratamiz(filtersiz datani)

In [20]:
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.23, random_state=99)

# filter ishlatilgan datani ajratamiz

In [22]:
x1_train,x1_test,y1_train,y1_test = train_test_split(x1,y1, test_size=0.2, random_state=100)

# ||---  algorithm lar bilan ishlashni boshlaymiz  ---||



# DECISION TREE(FILTERSIZ)

In [23]:
Rfe = RFE(DT_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

DT_Clas.fit(x_train[selected_features], y_train)
pred4 = DT_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred4))

0.9111448642490981


# DECISION TREE (FILTER ISHLATILGAN DATA BILAN)

In [25]:
Rfe = RFE(DT_Clas, n_features_to_select=10)
Rfe.fit(x1_train, y1_train)
selected_features = x1_train.columns[Rfe.support_]

DT_Clas.fit(x1_train[selected_features], y1_train)
pred4_1 = DT_Clas.predict(x1_test[selected_features])

print(accuracy_score(y1_test, pred4_1))

0.885589519650655


#  RANDOM FOREST (FILTER METHOD SIZ DATA BILAN)

In [26]:
Rfe = RFE(RF_Clas, n_features_to_select=10) 
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

RF_Clas.fit(x_train[selected_features], y_train)
pred3 = RF_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred3))

0.9813935826846402


# RANDOM FOREST(Filter methodi ishlatilgan data bilan)

In [27]:
Rfe = RFE(RF_Clas, n_features_to_select=10) 
Rfe.fit(x1_train, y1_train)
selected_features = x1_train.columns[Rfe.support_]

RF_Clas.fit(x1_train[selected_features], y1_train)
pred3_1 = RF_Clas.predict(x1_test[selected_features])

print(accuracy_score(y1_test, pred3_1))

0.9729257641921397


# XGBoost (Filter method ishlatilmagan data bilan)

In [31]:
Rfe = RFE(Xgb_Clas, n_features_to_select=10)
Rfe.fit(x_train, y_train)
selected_features = x_train.columns[Rfe.support_]

Xgb_Clas.fit(x_train[selected_features], y_train)
pred2 = Xgb_Clas.predict(x_test[selected_features])

print(accuracy_score(y_test, pred2))

ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [1 2 3 4 5 6]